In [60]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as pl
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel, Norm_Iso2DGaussianModel, DoG_Iso2DGaussianModel, CSS_Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter, Norm_Iso2DGaussianFitter, DoG_Iso2DGaussianFitter, CSS_Iso2DGaussianFitter

This notebook shows a simple example of prfpy use

# Creating stimulus object

In [61]:
screen_size_cm=40
screen_distance_cm=200
design_matrix=np.zeros((50,50,100))
design_matrix[:,20:30,40:50] = 1
TR=1
task_lengths=[100]
task_names=['test']
late_iso_dict={'test':np.arange(40)}
normalize_integral_dx=False

prf_stim = PRFStimulus2D(screen_size_cm=screen_size_cm,
                             screen_distance_cm=screen_distance_cm,
                             design_matrix=design_matrix,
                             TR=TR,
                             task_lengths=task_lengths,
                             task_names=task_names,
                             late_iso_dict=late_iso_dict,
                             normalize_integral_dx=normalize_integral_dx)

# Gaussian model fit

## Creating Gaussian model and fitter objects

In [99]:
#only needed if filtering predictions
filter_type='dc'
filter_params={"first_modes_to_remove":3,
                         "last_modes_to_remove_percent":0,
                         "window_length":50,
                         "polyorder":3,
                         "highpass":True,
                         "add_mean":True}

filter_predictions=False

hrf=[1,0.4,0]

data = 3*np.random.rand(50,100)+0.01*np.sum(design_matrix, axis=(0,1))-1.5
#some kind of surround
data[:10,30:40] -= 2.5
data[:10,50:60] -= 2
data = np.roll(data,12)

normalize_RFs=False


gg = Iso2DGaussianModel(stimulus=prf_stim,
                          hrf=hrf,
                          filter_predictions=filter_predictions,
                          filter_type=filter_type,
                          filter_params=filter_params,
                          normalize_RFs=normalize_RFs)

gf = Iso2DGaussianFitter(data=data, model=gg, n_jobs=8)

## Gaussian grid fit

In [100]:
ecc_grid=np.linspace(0,10,10)
polar_grid=np.linspace(-np.pi,np.pi,10)
size_grid=np.linspace(1,10,10)
verbose=False
n_batches=8
fixed_grid_baseline=0
gauss_grid_bounds=[(0,1000)] #bound on prf amplitudes (only positive)

hrf_1_grid=np.linspace(0,10,10)
hrf_2_grid=np.linspace(0,0,1)

gf.grid_fit(ecc_grid=ecc_grid,
                polar_grid=polar_grid,
                size_grid=size_grid,
                verbose=verbose,
                n_batches=n_batches,
                fixed_grid_baseline=fixed_grid_baseline,
                grid_bounds=gauss_grid_bounds)#,
           #hrf_1_grid=hrf_1_grid,
           #hrf_2_grid=hrf_2_grid)

In [101]:
gf.gridsearch_params[:,-1].max()

0.14288482069969177

## Gaussian Iterative Fit

In [102]:
rsq_threshold=0.01
verbose=True
gauss_bounds = [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0)]  # bold baseline
gauss_bounds += [(0.4,0.4),(0,0)] #hrf bounds. if want it fixed to some value, specify e.g. (4,4) (0,0)
constraints=None
xtol=1e-4
ftol=1e-4

gf.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                         bounds=gauss_bounds,
                         constraints=constraints,
                             xtol=xtol,
                             ftol=xtol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change i

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.63305D+02    |proj g|=  4.32010D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.72969D+02    |proj g|=  3.24007D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.81823D+02    |proj g|=  4.03588D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.49981D+02    |proj g|=  2.11916D-03
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =     

/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/User

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.86974D+02    |proj g|=  2.38742D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.85095D+02    |proj g|=  3.58114D-04

At iterate    1    f=  2.91783D+02    |proj g|=  8.52651D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      1      4      1     0     0   8.527D-04   2.918D+

/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':


...

/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returnin

  F =   349.57180407938370     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  2.84962D+02    |proj g|=  5.11591D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      1     11      1     0     0   5.116D-04   2.850D+02
  F =   284.96218044648236     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  2.99354D+02    |proj g|=  5.85487D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Na

/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search d


At iterate    1    f=  2.74239D+02    |proj g|=  2.67164D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      1     10      1     0     0   2.672D-04   2.742D+02
  F =   274.23880233532998     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  2.84051D+02    |proj g|=  4.66116D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final proj


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
[Parallel(n_jobs=8)]: Done  38 out of  38 | elapsed:    3.8s finished

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [103]:
gf.iterative_search_params

array([[ 0.00000000e+00,  0.00000000e+00,  4.00000000e+00,
         2.11915707e-03,  0.00000000e+00,  4.00000000e-01,
         0.00000000e+00,  1.27830968e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.15864981e-02,  0.00000000e+00,  4.00000000e-01,
         0.00000000e+00,  1.49407146e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.000000

# DN model fit

## Create model and fitter objects

In [105]:
stimulus=prf_stim
filter_type='dc'
filter_params={"first_modes_to_remove":3,
                         "last_modes_to_remove_percent":0,
                         "window_length":50,
                         "polyorder":3,
                         "highpass":True,
                         "add_mean":True}

filter_predictions=False                                     
normalize_RFs=False

use_previous_gaussian_fitter_hrf=False #if true, will use hrf result from gauss fit at the grid stage instead of doing a grid fit for it

gg_norm = Norm_Iso2DGaussianModel(stimulus=prf_stim,
                                    hrf=hrf,
                                    filter_predictions=filter_predictions,
                                    filter_type=filter_type,
                                    filter_params=filter_params,                                       
                                    normalize_RFs=normalize_RFs)

gf_norm = Norm_Iso2DGaussianFitter(data=data,
                                   model=gg_norm,
                                   n_jobs=8,
                                   previous_gaussian_fitter=gf,
                                  use_previous_gaussian_fitter_hrf=use_previous_gaussian_fitter_hrf) 

## DN model grid fit

In [107]:
norm_grid_bounds = [(0,1000),(0,1000)] #only prf amplitudes between 0 and 1000, only neural baseline values between 0 and 1000

gf_norm.grid_fit(surround_amplitude_grid=np.linspace(0,10,5),
             surround_size_grid=np.linspace(1,10,5),
             neural_baseline_grid=np.linspace(0,10,5),
             surround_baseline_grid=np.linspace(1,10,5),
             verbose=verbose,
             n_batches=8,
             rsq_threshold=rsq_threshold,
             fixed_grid_baseline=fixed_grid_baseline,
             grid_bounds=norm_grid_bounds),
            # hrf_1_grid=np.linspace(0,10,5),
            # hrf_2_grid=np.linspace(0,0,1))

Each batch contains approx. 5 voxels.


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: element

(None,)

In [110]:
gf_norm.gridsearch_rsq_mask.sum()

38

## DN Iterative Fit

In [111]:
norm_bounds =  [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0),  # bold baseline
                (0, 1000),  # surround amplitude
                (0.1, 3*5),  # surround size
                (0, 1000),  # neural baseline
                (1e-6, 1000)]  # surround baseline
norm_bounds += [(0.4,0.4),(0,0)]
constraints_norm = None

gf_norm.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                               bounds=norm_bounds,
                               constraints=constraints_norm,
                               xtol=xtol,
                               ftol=ftol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/Users/marcoaqil/prfpy/prfpy/model.py:560: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.26352D+02    |proj g|=  1.18040D+02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.10866D+02    |proj g|=  2.48583D+02

At iterate    1    f=  4.51826D+02    |proj g|=  6.28642D+00


At iterate    2    f=  4.48912D+02    |proj g|=  4.13970D+00
At iterate    1    f=  4.54931D+02    |proj g|=  2.20830D+01

At iterate    2    f=  4.50540D+02    |proj g|=  1.39173D+01

At iterate    3    f=  4.50060D+02    |proj g|=  6.93162D+00

At iterate    3    f=  4.48605D+02    |proj g|=  6.84525D-01

At iterate    4    f=  4.48600D+02    |proj g|=  5.06026D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = t

[Parallel(n_jobs=8)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   2 out of   2 | elapsed:    0.3s finished


# Crossvalidate

In [ ]:
test_data = 3*np.random.rand(50,100)+0.01*np.sum(design_matrix, axis=(0,1))-1.5
np.roll(test_data,6)

test_stimulus = prf_stim

single_hrf = False

gf.crossvalidate_fit(test_data=test_data,
                     test_stimulus=test_stimulus,
                     single_hrf=single_hrf)

gf_norm.crossvalidate_fit(test_data,
                        test_stimulus=test_stimulus,
                        single_hrf=single_hrf)